In [2]:
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

import numpy as np
import random
import io
import json
import re

In [3]:
# Loading the training data
path = '../data/manele-merged.json'
text = []
with io.open(path, encoding="utf-8") as f:
    file = json.load(f)
    for manea in file:
        for lyric in manea['lyrics']:
            text.append(lyric)
text = ''.join(text)


In [6]:
# Cleaning the text
text = text.lower()
to_replace = list('!"$&()*+/:;<=>@[]^_~{}#%\\|–…\ufeff\xa0§«»')
to_replace.append("'")
to_replace.append("refren")
to_replace.append("ref")
to_replace.append("x2")
to_replace.append("2x")
to_replace.append("florin salam")
to_replace.append("salam")
to_replace.append("bis")

for word in to_replace:
    text = text.replace(word, '')
    pass
text = re.sub('â|ă|а', 'a', text)
text = re.sub('í|î|ï|і|ἰ', 'i', text)
text = re.sub('ş|ș|ѕ', 's', text)
text = re.sub('ţ', 't', text)
text = re.sub('ν', 'v', text)
text = re.sub('в', 'b', text)
text = re.sub('е', 'e', text)
text = re.sub('к', 'k', text)
text = re.sub('м', 'm', text)
text = re.sub('н', 'h', text)
text = re.sub('о', 'o', text)
text = re.sub('р', 'p', text)
text = re.sub('с', 'c', text)
text = re.sub('т', 't', text)
text = re.sub('у', 'y', text)
text = re.sub('х', 'x', text)
text = re.sub('ј', 'j', text)


text = re.sub(r'\n\s*\n', '\n\n', text)
text = re.sub(r'\.{4,}', '...', text)
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print(chars)
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


Corpus length: 1584499
['\n', ' ', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'B', 'H', 'K', 'M', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Total chars: 47


In [ ]:

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [5]:
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
epochs = 15 
batch_size = 128 

model.fit(x, y, batch_size=batch_size, epochs=epochs)

Epoch 1/15
1036/1036 [==============================] - 96s 76ms/step - loss: 2.3902
Epoch 2/15
1036/1036 [==============================] - 79s 76ms/step - loss: 1.9048
Epoch 3/15
1036/1036 [==============================] - 82s 79ms/step - loss: 1.6867
Epoch 4/15
1036/1036 [==============================] - 79s 76ms/step - loss: 1.5471
Epoch 5/15
1036/1036 [==============================] - 79s 76ms/step - loss: 1.4329
Epoch 6/15
1036/1036 [==============================] - 78s 75ms/step - loss: 1.3327
Epoch 7/15
1036/1036 [==============================] - 77s 75ms/step - loss: 1.2426
Epoch 8/15
1036/1036 [==============================] - 79s 76ms/step - loss: 1.1619
Epoch 9/15
1036/1036 [==============================] - 78s 75ms/step - loss: 1.0868
Epoch 10/15
1036/1036 [==============================] - 78s 75ms/step - loss: 1.0200
Epoch 11/15
1036/1036 [==============================] - 78s 75ms/step - loss: 0.9596
Epoch 12/15
1036/1036 [==============================] - 78s 75

In [10]:
generated_length = 300
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5]:
    print("...Diversity:", diversity)

    generated = ""
    original_sentence = text[start_index : start_index + maxlen]
    sentence = original_sentence
    print('...Generating with seed:\n "' + sentence + '"')

    for i in range(generated_length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated:\n", original_sentence + generated)
    print()

...Diversity: 0.2
...Generating with seed:
 "u urata ii viata mea
buna sau rea eu tot vreau sa las ceva in urma mea
ca viata mea e si buna si rea"
...Generated:
 u urata ii viata mea
buna sau rea eu tot vreau sa las ceva in urma mea
ca viata mea e si buna si rea
sa mor de la mine nu te las la mine
si te iubesc da nu ma vrei sa iti dovedesc

ca tine nu te voi iubii

te-ai jucat cu mine
dar nu te vad nici atat de dor

mi-e dor, mi-e dor de ea
intr-o zi sa ma iubesti
nu mi-am promis lu teata viata mea
ca eu nu te intoarce
si nu te vad nici o sa mai fi sa fiu 

...Diversity: 0.5
...Generating with seed:
 "u urata ii viata mea
buna sau rea eu tot vreau sa las ceva in urma mea
ca viata mea e si buna si rea"
...Generated:
 u urata ii viata mea
buna sau rea eu tot vreau sa las ceva in urma mea
ca viata mea e si buna si rea
pe dragostea mea
sa pot sa uit ca voi traiesc
si sa te rau inima chiara

mi-e dor, mi-e dor de ea
dormi adintrimile si foricine

azi ma iubeste dumnezeu
ma doare mi-a pasasi

In [19]:
model.save("model-15-0.7753.h5")

In [15]:
# Save weights in hdf5 format
filepath="model-{epoch:02d}-{loss:.4f}.h5"
checkpoint = ModelCheckpoint(filepath + ".hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')